### This notebook is used while developing the json parser that will build the prompt for Gemma.

In [109]:
import json

# Preload the JSON data at module level
try:
    with open("../data/conversation-tree-v2.json", 'r', encoding='utf-8') as file:
        JSON_DATA = json.load(file)
except FileNotFoundError:
    print("❌ Error: File 'conversation-tree-simplified-with-audio.json' not found.")
    JSON_DATA = {}
except json.JSONDecodeError as e:
    print(f"❌ Error: Invalid JSON format - {e}")
    JSON_DATA = {}
except Exception as e:
    print(f"❌ Error: {e}")
    JSON_DATA = {}

In [110]:
def get_question(speaker_name, index=0):
    """
    Return message and audio for a speaker from the first level of questions.
    
    Args:
        speaker_name (str): Name of the speaker (e.g., 'student_1', 'teacher_1')
    
    Returns:
        dict: Dictionary containing 'message' and 'audio' for the speaker
    """
    results = []
    
    if 'questions' in JSON_DATA:
        initial_question = JSON_DATA['questions'][index]
        for question in initial_question['children']:
            print(question.get('speaker'))
            if question.get('speaker') == speaker_name:
                result = {
                    'message': question.get('message', ''),
                    'question': initial_question.get('message', ''),
                    'criteria': initial_question.get('criteria', ''),
                    'audio': question.get('audio link', ''),
                    'responses': question.get('responses', [])
                }
                return result
    return {
        'message':'Speaker not found',
        'audio': ''
    }

get_question("teacher_1",0)

teacher_1


{'message': "Good morning! Let's explore this together. In the context of projectile motion, how would you define a projectile?",
 'question': 'Within the topic of projectile motion, how would you describe a projectile?',
 'criteria': '',
 'audio': 'voice/teacher_1_question_1_0001.wav',
 'responses': [{'speaker': 'student_2',
   'message': "Okay, so a projectile is pretty much an object that's thrown or launched into the air, and then it just keeps moving, mostly because of gravity.",
   'tag': 'student_2_question_1_0001',
   'audio link': 'voice/student_2_question_1_0001.wav',
   'responses': [{'speaker': 'teacher_1',
     'message': 'You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.',
     'tag': 'teacher_1_question_1_0002',
     'audio link': 'voice/teacher_1_question_1_0002.wav',
     're

In [111]:
len(JSON_DATA['questions'])

3

In [92]:
get_question("teacher_1",1)

teacher_1


{'message': "Hi there! I'd like you to think about something. What mathematical function describes the path of a projectile?",
 'question': 'What mathematical function does a path of a projectile follow?',
 'criteria': '',
 'audio': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_2_0001.wav?download=true',
 'responses': [{'speaker': 'student_2',
   'message': 'Oh, is it a quadratic function?',
   'tag': 'student_2_question_2_0001',
   'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_2_0001.wav?download=true',
   'responses': [{'speaker': 'teacher_1',
     'message': 'You are very close! A quadratic function does describe the path. Now, think of the specific geometric shape that the graph of a quadratic function creates. What is that shape called?',
     'tag': 'teacher_1_question_2_0002',
     'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_2_0002.wav?downloa

In [93]:

def find_speaker_responses(data, speaker_name):
    """
    Find the first node that matches the speaker_name by traversing children recursively.
    
    Args:
        data: The JSON data structure (can be a dict or list)
        speaker_name: The speaker name to search for
        
    Returns:
        The first matching node or None if not found
    """
    if isinstance(data, dict):
        # get the responses of this node
        if 'responses' in data:
            data = data['responses']    
    if isinstance(data, list):        # Search through list items
        nodes = []
        for item in data:
            if 'responses' in item:
                for child in item['responses']:
                    if child.get('speaker') == speaker_name:
                        nodes.append(child)
        return nodes
    
    return None

start_node = get_question("teacher_1",0)
print(start_node)

next_nodes_1 = find_speaker_responses(start_node['responses'], "teacher_1")
print("next_nodes(1) length: ", len(next_nodes_1))
for node in next_nodes_1:
    print(">>>", node['message'])

teacher_1
{'message': "Good morning! Let's explore this together. In the context of projectile motion, how would you define a projectile?", 'question': 'Within the topic of projectile motion, how would you describe a projectile?', 'criteria': '', 'audio': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0001.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': "Okay, so a projectile is pretty much an object that's thrown or launched into the air, and then it just keeps moving, mostly because of gravity.", 'tag': 'student_2_question_1_0001', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_1_0001.wav?download=true', 'responses': [{'speaker': 'teacher_1', 'message': 'You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typ

In [94]:

#Get the second level
next_parent_2 = next_nodes_1[0]
print('next_parent(2)', next_parent_2['message'])
next_nodes_2 = find_speaker_responses(next_parent_2['responses'], "teacher_1")
print("next_nodes(2) length: ", len(next_nodes_2))
for node in next_nodes_2:
    print(">>>", node['message'])

next_parent(2) You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.
next_nodes(2) length:  3
>>> You are correct that gravity is a key factor. Now, remember that a projectile isn't just standing still. Think of how a projectile is initially set in motion. How would you describe a projectile, considering both its initial motion and the force acting on it?
>>> Within the topic of projectile motion, how would you describe a projectile?
>>> Remember that a projectile is an object that is set into motion, and consider the primary force that acts on it once it's in the air.


In [73]:
#Get the second level
next_parent_2 = next_nodes_1[0]
print('next_parent(2)', next_parent_2['message'])
next_nodes_2 = find_speaker_responses(next_parent_2, "teacher_1")
print("next_nodes(2) length: ", len(next_nodes_2))
for node in next_nodes_2:
    print(">>>", node['message'])

next_parent(2) Exactly! And what's special about these two components?
next_nodes(2) length:  2
>>> Perfect! That's the key insight of projectile motion.
>>> Brilliant observation! Yes, horizontal velocity stays constant while vertical motion accelerates due to gravity.


In [95]:
#Get the third level
next_parent_3 = next_nodes_2[0]
print('next_parent(3)', next_parent_3['message'])

if 'responses' in next_parent_3:  
    next_nodes_3 = find_speaker_responses(next_parent_3['responses'], "teacher_1")
    print("next_nodes(3) length: ", len(next_nodes_3))
    for node in next_nodes_3:
        print(">>>", node['message'])
else:
    print("No responses found for next_parent_3")
    # Let's move

next_parent(3) You are correct that gravity is a key factor. Now, remember that a projectile isn't just standing still. Think of how a projectile is initially set in motion. How would you describe a projectile, considering both its initial motion and the force acting on it?
next_nodes(3) length:  3
>>> That's correct!
>>> That's correct!
>>> Within the topic of projectile motion, how would you describe a projectile? Think of an object thrown into the air, and consider the primary force that pulls it back down.


In [96]:
# check the next question
start_node = get_question("teacher_1",1)
print(start_node)

next_nodes_1 = find_speaker_responses(start_node['responses'], "teacher_1")
print("next_nodes(1) length: ", len(next_nodes_1))
for node in next_nodes_1:
    print(">>>", node['message'])

teacher_1
{'message': "Hi there! I'd like you to think about something. What mathematical function describes the path of a projectile?", 'question': 'What mathematical function does a path of a projectile follow?', 'criteria': '', 'audio': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_2_0001.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': 'Oh, is it a quadratic function?', 'tag': 'student_2_question_2_0001', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_2_0001.wav?download=true', 'responses': [{'speaker': 'teacher_1', 'message': 'You are very close! A quadratic function does describe the path. Now, think of the specific geometric shape that the graph of a quadratic function creates. What is that shape called?', 'tag': 'teacher_1_question_2_0002', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_2_0002.wav?download=true', 'responses':

In [97]:
#Get the second level
next_parent_2 = next_nodes_1[0]
print('next_parent(2)', next_parent_2['message'])
next_nodes_2 = find_speaker_responses(next_parent_2['responses'], "teacher_1")
print("next_nodes(2) length: ", len(next_nodes_2))
for node in next_nodes_2:
    print(">>>", node['message'])

next_parent(2) You are very close! A quadratic function does describe the path. Now, think of the specific geometric shape that the graph of a quadratic function creates. What is that shape called?
next_nodes(2) length:  3
>>> Correct.
>>> Correct.
>>> Remember that the shape of the path of any object thrown or launched into the air, subject only to gravity, has a specific mathematical curve. What is that curve called?


In [98]:
question_index = 0
current_node = None
def build_response_prompt(speaker_name):

    """
    Return prompt to choose the best response from the node.
    """
    global current_node
    global question_index

    question_node = get_question(speaker_name, question_index)

    if current_node is None:
        current_node = get_question(speaker_name, question_index)


    response_nodes = find_speaker_responses(current_node['responses'], speaker_name)
    response_messages = []
    tags = []
    for node in response_nodes:
        response_messages.append(node['message'])
        tags.append(node['tag'])

    # build the prompt

    question_or_message = current_node.get("question") or current_node.get("message")

    prompt = f""" You are a tutor assessing a student's response. Choose the best response from the list of responses based on the criteria. Respond verbatim from the response.
    Question: {question_or_message}
    Criteria: {question_node['criteria']}
    Responses: {response_messages}
    """
    return (prompt, response_messages, tags)

response_prompt, response_messages, tags = build_response_prompt('teacher_1')
print(response_prompt)
print(response_messages)
print(tags)

teacher_1
teacher_1
 You are a tutor assessing a student's response. Choose the best response from the list of responses based on the criteria. Respond verbatim from the response.
    Question: Within the topic of projectile motion, how would you describe a projectile?
    Criteria: 
    Responses: ['You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.', "That's correct!", 'Within the topic of projectile motion, how would you describe a projectile?\n\nThink of how you would make something fly through the air.']
    
['You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.', "That's correct!", 'Within the to

In [99]:
current_node

{'message': "Good morning! Let's explore this together. In the context of projectile motion, how would you define a projectile?",
 'question': 'Within the topic of projectile motion, how would you describe a projectile?',
 'criteria': '',
 'audio': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0001.wav?download=true',
 'responses': [{'speaker': 'student_2',
   'message': "Okay, so a projectile is pretty much an object that's thrown or launched into the air, and then it just keeps moving, mostly because of gravity.",
   'tag': 'student_2_question_1_0001',
   'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_1_0001.wav?download=true',
   'responses': [{'speaker': 'teacher_1',
     'message': 'You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from grav

In [100]:
next_node = current_node['responses'][0]['responses'][0]
print(next_node)



{'speaker': 'teacher_1', 'message': 'You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.', 'tag': 'teacher_1_question_1_0002', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0002.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': "Okay, so it's mainly about gravity then, for the ideal one.", 'tag': 'student_2_question_1_0002', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_1_0002.wav?download=true', 'responses': [{'speaker': 'teacher_1', 'message': "You are correct that gravity is a key factor. Now, remember that a projectile isn't just standing still. Think of how a projectile is initially set in motion. How would you describe a projectile, considering both its initial motion 

In [101]:
current_node = next_node

response_prompt, response_messages, tags = build_response_prompt('teacher_1')
print(response_prompt)
print(response_messages)  
print(tags)

teacher_1
 You are a tutor assessing a student's response. Choose the best response from the list of responses based on the criteria. Respond verbatim from the response.
    Question: You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.
    Criteria: 
    Responses: ["You are correct that gravity is a key factor. Now, remember that a projectile isn't just standing still. Think of how a projectile is initially set in motion. How would you describe a projectile, considering both its initial motion and the force acting on it?", 'Within the topic of projectile motion, how would you describe a projectile?', "Remember that a projectile is an object that is set into motion, and consider the primary force that acts on it once it's in the air."]
    
["You are correct that gravity is a key factor. Now, r

In [102]:
def find_node_by_tag(tag):
    """
    Find the node with the given tag from the JSON data.
    
    Args:
        tag (str): The tag to search for
        
    Returns:
        dict: The node with the matching tag, or None if not found
    """
    def search_recursive(data):
        """Recursively search through the data structure for the tag."""
        if isinstance(data, dict):
            if data.get('tag') == tag:
                return data
            # Search in responses if they exist
            if 'responses' in data:
                for response in data['responses']:
                    result = search_recursive(response)
                    if result:
                        return result
            # Search in children if they exist
            if 'children' in data:
                for child in data['children']:
                    result = search_recursive(child)
                    if result:
                        return result
        elif isinstance(data, list):
            for item in data:
                result = search_recursive(item)
                if result:
                    return result
        return None
    
    # Start the search from the questions
    if 'questions' in JSON_DATA:
        for question in JSON_DATA['questions']:
            result = search_recursive(question)
            if result:
                return result
    
    return None

node = find_node_by_tag('teacher_1_question_1_0001')
print(node)

{'speaker': 'teacher_1', 'message': "Good morning! Let's explore this together. In the context of projectile motion, how would you define a projectile?", 'tag': 'teacher_1_question_1_0001', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0001.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': "Okay, so a projectile is pretty much an object that's thrown or launched into the air, and then it just keeps moving, mostly because of gravity.", 'tag': 'student_2_question_1_0001', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_1_0001.wav?download=true', 'responses': [{'speaker': 'teacher_1', 'message': 'You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.', 'tag': 'teacher_1_question_1_0

In [107]:

node = find_node_by_tag('student_2_question_1_0002')
print(node)

{'speaker': 'student_2', 'message': "Okay, so it's mainly about gravity then, for the ideal one.", 'tag': 'student_2_question_1_0002', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_1_0002.wav?download=true', 'responses': [{'speaker': 'teacher_1', 'message': "You are correct that gravity is a key factor. Now, remember that a projectile isn't just standing still. Think of how a projectile is initially set in motion. How would you describe a projectile, considering both its initial motion and the force acting on it?", 'tag': 'teacher_1_question_1_0003', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0003.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': "Okay, so it's like anything that gets launched or thrown, and then gravity is the main force pulling it down.", 'tag': 'student_2_question_1_0003', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve

In [106]:

node = find_node_by_tag('student_2_question_1_0001')
print(node)

{'speaker': 'student_2', 'message': "Okay, so a projectile is pretty much an object that's thrown or launched into the air, and then it just keeps moving, mostly because of gravity.", 'tag': 'student_2_question_1_0001', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/student_2_question_1_0001.wav?download=true', 'responses': [{'speaker': 'teacher_1', 'message': 'You are correct that a projectile is an object that is thrown or launched into the air and that gravity affects its motion. Remember that in the ideal definition of projectile motion, all other forces apart from gravity are typically disregarded.', 'tag': 'teacher_1_question_1_0002', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0002.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': "Okay, so it's mainly about gravity then, for the ideal one.", 'tag': 'student_2_question_1_0002', 'audio link': 'https://huggingface.co/datasets/it

In [108]:

node = find_node_by_tag('teacher_1_question_1_0015')
print(node)

{'speaker': 'teacher_1', 'message': "That's correct!", 'tag': 'teacher_1_question_1_0015', 'audio link': 'https://huggingface.co/datasets/itsybitsci/audio/resolve/main/teacher_1_question_1_0015.wav?download=true', 'responses': []}
